In [1]:
import subprocess
import sys
from tqdm import tqdm
import time
import os
os.environ['HYDRA_FULL_ERROR'] = '1'  
# Запуск обучения с отслеживанием лога
def run_training():
    command = [sys.executable, "train.py"]
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
    
    tqdm_bar = tqdm(total=100, desc="Обучение модели", unit="%", dynamic_ncols=True)
    
    for line in iter(process.stdout.readline, ""):
        tqdm_bar.update(1)  # Каждая строка лога увеличивает прогресс
        tqdm_bar.set_postfix_str(line.strip())  # Отображение текущего состояния
        time.sleep(0.1)

    process.stdout.close()
    process.wait()
    tqdm_bar.close()

# Запускаем обучение
run_training()

Обучение модели:   4%|▍         | 4/100 [00:03<01:35,  1.00%/s, FileNotFoundError: [WinError 2] РќРµ СѓРґР°РµС‚СЃСЏ РЅР°Р№С‚Рё СѓРєР°Р·Р°РЅРЅС‹Р№ С„Р°Р№Р»: 'detection']


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob

# Функция для загрузки логов PyTorch Lightning
def load_logs(log_dir="lightning_logs"):
    log_files = sorted(glob.glob(f"{log_dir}/version_*/metrics.csv"), key=os.path.getctime)
    
    if not log_files:
        raise FileNotFoundError("Не найдено файлов с логами. Запустите обучение.")
    
    log_file = log_files[-1]  # Берём самый последний лог
    df = pd.read_csv(log_file)
    return df

# Загружаем данные
df = load_logs()

# Построение графиков
plt.figure(figsize=(12, 5))

# Лосс
plt.subplot(1, 2, 1)
plt.plot(df["epoch"], df["train_loss"], label="Train Loss", color="blue")
plt.plot(df["epoch"], df["val_loss"], label="Validation Loss", color="red")
plt.xlabel("Эпоха")
plt.ylabel("Loss")
plt.title("Изменение Loss во время обучения")
plt.legend()

# Точность (val_mAP)
plt.subplot(1, 2, 2)
plt.plot(df["epoch"], df["val_mAP"], label="Validation mAP", color="green")
plt.xlabel("Эпоха")
plt.ylabel("mAP")
plt.title("Изменение точности (mAP) на валидации")
plt.legend()

plt.show()